### Defining imports

In [1]:
import numpy as np 
import pandas as pd
import pickle
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

### Loading the dataset

After dataset loading, we remove duplicated or NaN rows.

52 rows are identified as duplicate. No NaN is found.

In [2]:
path = "../datasets/train.csv"
df = pd.read_csv(path)

print(len(df))

df = df.drop_duplicates()

print(len(df))

df = df.dropna()

print(len(df))

252175
252123
252123


******

### Dataset splitting

In [3]:
X = df.drop('Year', axis=1)
y = df['Year']

### Outlier removal via winsorization
When cell values are respectively below or above the 5th lower or upper percentile, we change its value to the treshold.

In [4]:
def winsorize_outliers(df, column, lower_limit, upper_limit):
    df[column] = np.where(df[column] < lower_limit, lower_limit, df[column])
    df[column] = np.where(df[column] > upper_limit, upper_limit, df[column])
    return df

lower_limit = X.quantile(0.05, axis=0)
upper_limit = X.quantile(0.95, axis=0)

for col in X.columns:
    X = winsorize_outliers(X.copy(), col, lower_limit[col], upper_limit[col])

### Dataset splitting
We split both `X` and `y` into training and validation sets, before converting them back into dataframes

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

print("----Training Set----")
print("X_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)

print("\n----Test Set----")
print("X_test Shape:", X_test.shape)
print("y_test Shape:", y_test.shape)

----Training Set----
X_train Shape: (201698, 90)
y_train Shape: (201698, 1)

----Test Set----
X_test Shape: (50425, 90)
y_test Shape: (50425, 1)


### MinMax Scaling
A MinMax scaler is used and saved for later usage. The StandardScaler and L1/L2 normalization were also tried but gave worse results on the 4 models.

In [6]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#normalization l1
#X_train = normalize(X_train, norm="l1")
#X_test = normalize(X_test, norm="l1")

#normalization l2
#X_train = normalize(X_train, norm="l2")
#X_test = normalize(X_test, norm="l2")

#standardization
#scalerStandard = StandardScaler()

#X_train = scalerStandard.fit_transform(X_train)
#X_test = scalerStandard.transform(X_test)

file = open("scaler.save","wb")
pickle.dump(scaler, file)
file.close()

### PCA
A principal component analysis is saved in `pca.save` file. A variance of 99% was used, if decreased it would have a heavy impact on the results. In this way, however, the results remain approximately unchanged compared to the approach without PCA but the dimensionality of the dataset is slightly reduced.

In [7]:
pca = PCA(0.99)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

file = open("pca.save","wb")
pickle.dump(pca, file)
file.close()

*****

### Machine Learning
We will use four models:
* Linear Regression
* K-Neighbours Regression
* Support Vector Regression
* Random Forest

For each one of them we show performances on validation set.

### Linear Regression
A linear regression model is instantiated, trained on training data and tested on testing data. Finally, the metrics are calculated.

In [8]:
linear_reg = LinearRegression()

linear_reg.fit(X_train, y_train)

y_pred_linear = linear_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred_linear)
r2 = r2_score(y_test, y_pred_linear)
mae = mean_absolute_error(y_test, y_pred_linear)
mape = mean_absolute_percentage_error(y_test, y_pred_linear)

perf = {"model": "lr", "mse": mse, "mae": mae, "mape": mape, "r2score": r2}
print(perf)

file = open("lr.save","wb")
pickle.dump(linear_reg, file)
file.close()

{'model': 'lr', 'mse': 81.41203699814004, 'mae': 6.524166889778804, 'mape': 0.003274426855344165, 'r2score': 0.2530491834152526}


### K-Neighbours Regressor
We implemented a `GridSearch` with various parameters.

After the training, we pick the best performing model, we print metrics and we save it in `knr.save`

The best performances were obtained with
* 30 neighbors
* `distance` weighting method
* p = 1

In [9]:
knn = KNeighborsRegressor()

param_grid_knn = {
    'n_neighbors': [1, 5, 10, 15, 20, 30, 50],  
    'weights': ['uniform', 'distance'],  
    'p': [1,2]
}

grid_search_knn = GridSearchCV(estimator=knn, param_grid=param_grid_knn, refit=True, cv=3, scoring='neg_mean_squared_error', verbose=3, n_jobs=-1)

grid_search_knn.fit(X_train, y_train)

best_knn = grid_search_knn.best_estimator_

print("Migliori parametri:", grid_search_knn.best_params_)

y_pred_knn = best_knn.predict(X_test)

mse = mean_squared_error(y_test, y_pred_knn)
r2 = r2_score(y_test, y_pred_knn)
mae = mean_absolute_error(y_test, y_pred_knn)
mape = mean_absolute_percentage_error(y_test, y_pred_knn)

perf = {"model": "knr", "mse": mse, "mae": mae, "mape": mape, "r2score": r2}
print(perf)

file = open("knr.save","wb")
pickle.dump(best_knn, file)
file.close()

Fitting 3 folds for each of 28 candidates, totalling 84 fits
Migliori parametri: {'n_neighbors': 30, 'p': 1, 'weights': 'distance'}
{'model': 'knr', 'mse': 81.36319668833968, 'mae': 6.656362605061602, 'mape': 0.0033404550622397416, 'r2score': 0.25349729048433944}


### Random Forest Regressor

We also used `GridSearch` for RF model.

Fewer configurations were tried at once due to longer execution times.

As above, the model with the best performances is stored in a file named `rf.save`

The best performances were obtained with
* 500 estimators
* 100 maximum depth

In [10]:
rf = RandomForestRegressor()

param_grid = {
    'n_estimators': [100, 300, 500],  
    'max_depth': [None, 50, 100],  
    #'min_samples_split': [2, 5], #too much time
    #'min_samples_leaf': [1, 2],  #too much time
}

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, refit=True, verbose=3, n_jobs=-1)

grid_search_rf.fit(X_train, y_train.to_numpy().ravel())

best_rf = grid_search_rf.best_estimator_

print("Migliori parametri:", grid_search_rf.best_params_)

y_pred_rf = best_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
mape = mean_absolute_percentage_error(y_test, y_pred_rf)

perf = {"model": "rf", "mse": mse, "mae": mae, "mape": mape, "r2score": r2}
print(perf)

file = open("rf.save","wb")
pickle.dump(best_rf, file)
file.close()

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Migliori parametri: {'max_depth': 100, 'n_estimators': 500}
{'model': 'rf', 'mse': 84.76421254108078, 'mae': 6.849759762022807, 'mape': 0.0034368472893292687, 'r2score': 0.22229316315755754}


### Support Vector Regressor
This last training also contains a `GridSearch`:

The chosen parameters were:
* `C=10`
* Radial basis function kernel

In [11]:
svm = SVR()

param_grid_svm = {
    'kernel': ['linear', 'rbf'], #poly too much time
    'C': [0.1, 1, 10], #0.01 too much time
}

grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, scoring='neg_mean_squared_error', cv=3, refit=True, verbose=3, n_jobs=-1)

grid_search_svm.fit(X_train, y_train.to_numpy().ravel())

best_svr = grid_search_svm.best_estimator_

print("Migliori parametri:", grid_search_svm.best_params_)

y_pred_svm = best_svr.predict(X_test)

mse = mean_squared_error(y_test, y_pred_svm)
r2 = r2_score(y_test, y_pred_svm)
mae = mean_absolute_error(y_test, y_pred_svm)
mape = mean_absolute_percentage_error(y_test, y_pred_svm)

perf = {"model": "svr", "mse": mse, "mae": mae, "mape": mape, "r2score": r2}
print(perf)

file = open("svr.save","wb")
pickle.dump(best_svr, file)
file.close()

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Migliori parametri: {'C': 10, 'kernel': 'rbf'}
{'model': 'svr', 'mse': 72.59192773264266, 'mae': 5.679693741821042, 'mape': 0.0028536365200389907, 'r2score': 0.33397318508813145}
